In [23]:
import pandas as pd
import collections
import os
# import nltk

In [60]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english')) 
punc = set([',', '-', '/', '.', '  ', 'university', 'research', 'center', 'institute'])
stop_words.update(punc)
'of' in stop_words

True

In [64]:
from fuzzywuzzy import fuzz

key = 'University of Illinois Urbana - Champaign'
value = 'University of Illinois at Urbana, Champaign'

def clean(name):
    name = name.lower()
    clean_name = []
    tok = word_tokenize(name)
    for token in tok:
        if token in stop_words:
            continue
        else:
            clean_name.append(token)
    return " ".join(clean_name)


def fuzzy(key, value):
    key = clean(key)
    value = clean(value)
    fuzz_score = fuzz.token_set_ratio(key, value)
    return fuzz_score

fuzzy('University of British Columbia', 'U British Columbia')

100

In [74]:
#creating a csv file to contain all the conferences TPC details

ipsn_tpc = pd.read_csv('data/Clean_TPC/ipsn_clean.csv')
ipsn_auth = pd.read_csv('data/Clean_Authors/ipsn_authors.csv')
mobicom_tpc = pd.read_csv('data/Clean_TPC/mobicom_clean.csv')
mobihoc_tpc = pd.read_csv('data/Clean_TPC/mobihoc_clean.csv')
sensys_tpc = pd.read_csv('data/Clean_TPC/sensys_clean.csv')
sigcomm_tpc = pd.read_csv('data/Clean_TPC/sigcomm_clean.csv')

In [76]:
keys = set(ipsn_tpc['University/Organization'])
uni_dict = collections.defaultdict(list)
for key in keys:
    uni_dict[key] = []
    
# uni_dict

In [93]:
unis = set(ipsn_auth['University/Organization'])
tup_list = []
for uni in unis:
    for key in uni_dict.keys():
        try:
            fuzz_score = fuzzy(uni, key)
            if fuzz_score > 95:
#                 print(key,'****', uni)
                tup_list.append((uni, key))
#                 uni_dict[key].append(uni)
                break
            else:
                continue
        except:
            continue

In [126]:
author_name = []
publications = []
auth_univ = []

tpc_univ = []
tpc_name = []

for tup in tup_list[:10]:

    auth_uni, tpc_uni = tup

    
    #extract author name and publication for auth_uni
    auth_df = ipsn_auth[ipsn_auth['University/Organization'] == auth_uni]
    author_name.extend(list(auth_df['Author_Name']))
    publications.extend(list(auth_df['Paper_title']))
    
    tpc_df = ipsn_tpc[ipsn_tpc['University/Organization'] == tpc_uni]
    tpc_name.extend(list(tpc_df['Name']))
    
    if len(auth_uni) < len(tpc_uni):
        auth_univ.extend([auth_uni])
        tpc_univ.extend([auth_uni])
    else:
        auth_univ.extend([tpc_uni])
        tpc_univ.extend([tpc_uni])
        
a = pd.DataFrame(list(zip(author_name, auth_univ, publications)), columns = ['Author_Name', 'University/Organization', 'Paper'])
b = pd.DataFrame(list(zip(tpc_name, tpc_univ)), columns = ['TPC_Name', 'University/Organization'])

In [130]:
a

,Author_Name,University/Organization,Paper
0,António Gonga,KTH Royal Institute of Technology,Multi-Channel Communication vs. Adaptive Routi...
1,Pablo Soldati,Intel,Multi-Channel Communication vs. Adaptive Routi...
2,Mikael Johansson,Queensland University of Technology,Multi-Channel Communication vs. Adaptive Routi...
3,Euhanna Ghadimi,Institute of Computing Technology,"Low Power, Low Delay: Opportunistic Routing Me..."
4,Mikael Johansson,University of Utah,"Low Power, Low Delay: Opportunistic Routing Me..."
5,Phillip B. Gibbons,Michigan State,Communicating via Fireflies: Geographic Routin...
6,Jie Xie,Hong Kong University of Science and Technology,Long-Term Monitoring of Cane-Toads Using Acous...
7,Jinglan Zhang,University of Oxford,Long-Term Monitoring of Cane-Toads Using Acous...
8,Michael W Towsey,University of Colorado,Long-Term Monitoring of Cane-Toads Using Acous...
9,Paul Roe,Imperial College,Long-Term Monitoring of Cane-Toads Using Acous...


In [128]:
b

,TPC_Name,University/Organization
0,Karl Henrik Johansson,KTH Royal Institute of Technology
1,Phil Gibbons,Intel
2,Ben Greenstein,Queensland University of Technology
3,Lama Nachman,Institute of Computing Technology
4,Phil Gibbons,University of Utah
5,Lama Nachman,Michigan State
6,Mark Yarvis,Hong Kong University of Science and Technology
7,Lakshman Krishnamurthy,University of Oxford
8,Peter Corke,University of Colorado
9,Cui Li,Imperial College


In [104]:
ipsn_auth[ipsn_auth['University/Organization'] == 'Intel Research Pittsburgh']

,Author_Name,University/Organization,Paper_title,Keywords,Year,Conference
1705,Phillip B. Gibbons,Intel Research Pittsburgh,Communicating via Fireflies: Geographic Routin...,"sleep-scheduling algorithm, geographic routing",2007,ipsn


In [91]:
fuzzy('Washington University in St. Louis', 'University of Washington')

100

In [72]:
grouping = collections.defaultdict(list)
captured_uni = set()
for uni in set(ipsn_tpc['University/Organization']):
    if uni not in grouping.keys() and uni not in captured_uni:
        for key in grouping.keys():
            fuzz_score = fuzzy(key, value)
            #append to the key value
            if fuzz_score > 60:
                grouping[key].append(uni)
            else:
                continue
        grouping[uni] = []
        captured_uni.add(uni)

In [73]:
grouping

defaultdict(list,
            {'SRI International': [],
             'University of Houston, USA': [],
             'Crossbow': [],
             'Intel Labs': [],
             'Institute of Computing Technology, Chinese Academy of Sciences': [],
             'KTH Royal Institute of Technology': [],
             'Maryland': [],
             'University of California San Diego': [],
             'Swedish Institute of Computer Science, Sweden': [],
             'University of California Irvine': [],
             'SUPSI Lugano, Switzerland': [],
             'National Science Foundation': [],
             'ETH, Zurich': [],
             'University College Cork, Ireland': [],
             'University of Duisburg-Essen and Fraunhofer IAIS': [],
             'University of Szeged, Hungary': [],
             'CISTER - ISEP/IPP, Portugal': [],
             'Hitachi Research': [],
             'UFMG, Brazil': [],
             'UNSW and CSIRO': [],
             'Purdue': [],
             'Duke U

In [26]:
full_tpc = pd.concat([ipsn_tpc, mobicom_tpc, mobihoc_tpc, sensys_tpc, sigcomm_tpc])
uni_list = set(full_tpc['University/Organization'])

In [27]:
full_tpc.to_csv('data/Clean_TPC/full_tpc.csv', header = True, index = False)

In [29]:
uni_list

{nan,
 'Seoul',
 'AON Networks',
 'Vanderbilt University, USA',
 'University Washington',
 'Princeton & MSR',
 'New Jersey Institute of Technology',
 'Indian Institute of Technology Chennai',
 'Texas A&M University, USA',
 'University of Michigan',
 'Switzerland',
 'Technicolor Paris Research Center',
 'Ecole Polytechnique',
 'University of Illinois Urbana-Champaign',
 'Vanderbilit/ISIS',
 'ASU',
 'University of New South Wales, Australia',
 'Ben-GurionUniversity of the Negev',
 'Waterloo',
 'University of Bonn',
 'National Academies',
 'Heriot-Watt University',
 'KAIST',
 'NTT Do',
 'Lehigh University',
 'University of Pisa',
 'Ericsson',
 'California Institute of Technology',
 'AT&T Research',
 'Columbia University and VMware',
 'Univerisity of California San Diego',
 'Amherst',
 'Rice University',
 'UW-Madison',
 'Brown University',
 'University of Chicago',
 'Georgia Tech',
 'UCL',
 'UMass-Amherst, USA',
 'Stony Brook University',
 'Purdue University, USA',
 'Ohio State University'

In [142]:
for uni in uni_list:
    if type(uni) == str:
        if uni.lower().find('nec') > -1:
            print(uni)

NEC Labs America
NEC Laboratories
NEC
NEC Labs Princeton
NEC Laboratories America
NEC Labs
NEC Research Europe
NEC LABS


In [11]:
uni_df = {}
for uni in uni_list:
#     print(uni)
#     print(full_tpc[full_tpc['University/Organization'] == uni])
    uni_df[uni] = full_tpc[full_tpc['University/Organization'] == uni]

In [23]:
uni = list(uni_df.keys())
# uni = ['app', 'zwe', 'bsd']
uni

[nan,
 'Seoul',
 'AON Networks',
 'Vanderbilt University, USA',
 'University Washington',
 'Princeton & MSR',
 'New Jersey Institute of Technology',
 'Indian Institute of Technology Chennai',
 'Texas A&M University, USA',
 'University of Michigan',
 'Switzerland',
 'Technicolor Paris Research Center',
 'Ecole Polytechnique',
 'University of Illinois Urbana-Champaign',
 'Vanderbilit/ISIS',
 'ASU',
 'University of New South Wales, Australia',
 'Ben-GurionUniversity of the Negev',
 'Waterloo',
 'University of Bonn',
 'National Academies',
 'Heriot-Watt University',
 'KAIST',
 'NTT Do',
 'Lehigh University',
 'University of Pisa',
 'Ericsson',
 'California Institute of Technology',
 'AT&T Research',
 'Columbia University and VMware',
 'Univerisity of California San Diego',
 'Amherst',
 'Rice University',
 'UW-Madison',
 'Brown University',
 'University of Chicago',
 'Georgia Tech',
 'UCL',
 'UMass-Amherst, USA',
 'Stony Brook University',
 'Purdue University, USA',
 'Ohio State University'

In [14]:
set(uni_df['California Institute of Technology']['Name'])

{'Jayakrishnan Nair', 'Jayakrishnan Unnikrishnan Nair', 'Steven Low'}